In [1]:
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
Image.MAX_IMAGE_PIXELS = None
import openslide as os
import pandas as pd
import json

In [2]:
# Be able to load python scripts
import sys
from pathlib import Path
module_path = Path('.').resolve().parent
if str(module_path) not in sys.path:
    sys.path.append(str(module_path))

In [2]:
from src.visualization.visualize import draw_heatmap_slide
from src.config import SLIDE_DIR, RESULT_DIR, ANNOT_DIR

ModuleNotFoundError: No module named 'src'

In [4]:
EXPERIMENT_ID = 'BIG-TILE-0253-43ac-acbc-bbf75ff6a58f'
NEW_EXPERIMENT_ID = 'IMAGE-RECONSTRUCTION-0000-0000-0000-bbf75ff6a58f'
CENTER_SIZE = 256

In [5]:
# Create new directory for new experiment
(RESULT_DIR / NEW_EXPERIMENT_ID).mkdir(exist_ok=True)
(RESULT_DIR / NEW_EXPERIMENT_ID / 'test' / 'figs').mkdir(exist_ok=True, parents=True)
(RESULT_DIR / NEW_EXPERIMENT_ID / 'test' / 'probas').mkdir(exist_ok=True, parents=True)

In [6]:
# Load summary.json
with (RESULT_DIR / EXPERIMENT_ID / 'summary.json').open('r') as summary_json_fp:
    summary_json = json.load(summary_json_fp)
    
# Get needed data from summary.json
tile_size = summary_json['params']['data']['meta']['tile_size']
scaling = 2**summary_json['params']['data']['meta']['level']

In [ ]:
# Start transforming images
total_idx = len(list(summary_json['test']['summary'].keys()))
for cur_idx, test_slide in enumerate(summary_json['test']['summary']):
    print('Processing {slide_name}   {cur_idx}/{total_idx}\r'.format(slide_name=test_slide, cur_idx=cur_idx+1, total_idx=total_idx))
    fig_fp = summary_json['test']['summary'][test_slide]['figures']['heatmap']
    fig_fp = fig_fp.replace(EXPERIMENT_ID, NEW_EXPERIMENT_ID)
    results_fp = summary_json['test']['summary'][test_slide]['results']
    df = pd.read_csv(results_fp, sep=';', index_col=0)
    
    draw_heatmap_slide(slide_name=test_slide, slide_dir=SLIDE_DIR, level=0, \
                       tile_size=tile_size*scaling, \
                       probas=df, \
                       fig_filepath=fig_fp, \
                       center_size=CENTER_SIZE*scaling)